In [1]:
"""
ML LEARNING JOURNEY - DAY 13
Week: 2 of 24
Day: 13 of 168
Date: October 27, 2025
Topic: Fine-tuning YOLO & Model Optimization
Overall Progress: 7.7%

Today's Goals:
- Understand hyperparameter tuning for YOLO
- Learn training configuration best practices
- Understand evaluation metrics (mAP, precision, recall)
- Learn model comparison methodology
- Understand optimization strategies
"""
print("=" * 60)
print("DAY 13: FINE-TUNING YOLO & OPTIMIZATION")
print("=" * 60)
print("\n🎯 Goal: Learn how to optimize YOLO models!\n")

DAY 13: FINE-TUNING YOLO & OPTIMIZATION

🎯 Goal: Learn how to optimize YOLO models!



In [2]:
# ============================================================
# IMPORTS & SETUP
# ============================================================
from ultralytics import YOLO
import os
from pathlib import Path
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from datetime import datetime

print("✅ All imports successful!")
print("=" * 60)

✅ All imports successful!


In [3]:
# ============================================================
# EXERCISE 1.1: UNDERSTANDING MODEL ITERATION
# ============================================================
print("\n" + "="*60)
print("EXERCISE 1.1: Model Iteration Strategy")
print("="*60)

print("""
WHY ITERATE?
══════════════════════════════════════════════════════════
First training rarely gives perfect results!

Machine Learning is an iterative process:
v1 → Analyze → v2 → Analyze → v3 → Production

TYPICAL ITERATION CYCLE:
══════════════════════════════════════════════════════════

VERSION 1 (Baseline):
──────────────────────────────────────────────────────────
Purpose: Establish baseline performance
Configuration:
- Small epochs (10-20)
- Default hyperparameters
- Quick training (~5 minutes)

Expected Result:
- Low accuracy (20-40%)
- Identifies major issues
- Proves pipeline works

Example: 10 epochs, 66 images → 17.1% mAP
✅ Success: Model trains without errors
❌ Problem: Too few epochs, needs more training

──────────────────────────────────────────────────────────

VERSION 2 (Optimized):
──────────────────────────────────────────────────────────
Purpose: Improve with better hyperparameters
Configuration:
- More epochs (50-100)
- Tuned learning rate
- Better augmentation
- Longer training (~10-15 minutes)

Expected Result:
- Medium accuracy (50-70%)
- Better class detection
- Identifies data needs

Example: 50 epochs, 66 images → 48.5% mAP
✅ Success: 3x improvement from v1!
❌ Problem: Still needs more data for 70%+

──────────────────────────────────────────────────────────

VERSION 3 (Production):
──────────────────────────────────────────────────────────
Purpose: Reach production-ready performance
Configuration:
- More data (3-4x increase)
- Bigger model (YOLOv8n → YOLOv8s)
- Max epochs (100+)
- Full training (~30 minutes)

Expected Result:
- High accuracy (70-90%)
- Production-ready
- Deployable model

Example: 100 epochs, 246 images → 75.1% mAP
✅ Success: Production target achieved!
✅ Ready: Deploy to real-world application

══════════════════════════════════════════════════════════
KEY INSIGHT:
══════════════════════════════════════════════════════════
DON'T try to make v1 perfect!

Instead:
1. v1: Quick baseline (identify issues)
2. v2: Address issues (improve setup)
3. v3: Scale up (reach production)

This is FASTER than trying to get v1 perfect!
══════════════════════════════════════════════════════════
""")

print("✅ Exercise 1.1 Complete!")
print("\n📝 Key Takeaway: Iteration > Perfection on first try!")


EXERCISE 1.1: Model Iteration Strategy

WHY ITERATE?
══════════════════════════════════════════════════════════
First training rarely gives perfect results!

Machine Learning is an iterative process:
v1 → Analyze → v2 → Analyze → v3 → Production

TYPICAL ITERATION CYCLE:
══════════════════════════════════════════════════════════

VERSION 1 (Baseline):
──────────────────────────────────────────────────────────
Purpose: Establish baseline performance
Configuration:
- Small epochs (10-20)
- Default hyperparameters
- Quick training (~5 minutes)

Expected Result:
- Low accuracy (20-40%)
- Identifies major issues
- Proves pipeline works

Example: 10 epochs, 66 images → 17.1% mAP
✅ Success: Model trains without errors
❌ Problem: Too few epochs, needs more training

──────────────────────────────────────────────────────────

VERSION 2 (Optimized):
──────────────────────────────────────────────────────────
Purpose: Improve with better hyperparameters
Configuration:
- More epochs (50-100)
- Tun

In [4]:
# ============================================================
# EXERCISE 1.2: HYPERPARAMETER TUNING
# ============================================================
print("\n" + "="*60)
print("EXERCISE 1.2: Understanding Hyperparameters")
print("="*60)

print("""
YOLO HYPERPARAMETERS EXPLAINED:
══════════════════════════════════════════════════════════

1. MODEL SELECTION
──────────────────────────────────────────────────────────
model: 'yolov8n.pt' / 'yolov8s.pt' / 'yolov8m.pt'

Options:
- YOLOv8n (nano): 3.2M params, fastest, good for learning
- YOLOv8s (small): 11M params, balanced, production-ready
- YOLOv8m (medium): 25M params, slower, highest accuracy

When to upgrade:
- Start with 'n' (nano) for quick experiments
- Upgrade to 's' (small) when data quality is good
- Use 'm' (medium) only if 's' plateaus below target

══════════════════════════════════════════════════════════

2. TRAINING DURATION
──────────────────────────────────────────────────────────
epochs: 50-100
patience: 20-30

Epochs: How many times to see full dataset
- Too few (10): Underfitting, poor results
- Good (50-100): Balanced learning
- Too many (200+): Overfitting risk, waste time

Patience: Early stopping tolerance
- Stop if no improvement after N epochs
- Prevents wasting time
- Typical: 20-30 epochs patience

Rule of thumb:
- Small dataset (<100 images): 50 epochs
- Medium dataset (100-500): 100 epochs
- Large dataset (500+): 50-100 epochs (converges faster)

══════════════════════════════════════════════════════════

3. LEARNING RATE
──────────────────────────────────────────────────────────
lr0: 0.01        # Initial learning rate
lrf: 0.01        # Final learning rate
warmup_epochs: 3  # Warmup period

Learning rate controls step size:
- Too high (0.1+): Unstable, might not converge
- Good (0.001-0.01): Balanced learning
- Too low (0.0001): Too slow, wastes time

Warmup: Gradual LR increase at start
- Prevents early instability
- 3-5 epochs typical
- Helps with transfer learning

══════════════════════════════════════════════════════════

4. BATCH SIZE
──────────────────────────────────────────────────────────
batch: 8 / 16 / 32

Batch size: Images per training step
- Larger batch = more stable gradients, but needs more memory
- Smaller batch = more updates, but noisier

Recommendations:
- CPU: 8-16 (limited memory)
- GPU (8GB): 16-32
- GPU (16GB+): 32-64

Too small (<8): Training unstable
Too large (>64): Diminishing returns

══════════════════════════════════════════════════════════

5. AUGMENTATION
──────────────────────────────────────────────────────────
hsv_h: 0.015      # Hue variation
hsv_s: 0.7        # Saturation
hsv_v: 0.4        # Brightness
degrees: 0.0      # Rotation
translate: 0.1    # Translation
scale: 0.5        # Scale variation
fliplr: 0.5       # Horizontal flip probability
mosaic: 1.0       # Mosaic augmentation
mixup: 0.0-0.15   # Mixup augmentation

Purpose: Increase effective dataset size

Conservative (safer):
- degrees: 0 (no rotation for upright objects)
- fliplr: 0.5 (50% chance horizontal flip)
- mosaic: 1.0 (always on, very effective)

Aggressive (more variation):
- degrees: 10-15 (rotate for varied poses)
- mixup: 0.15 (blend images)
- copy_paste: 0.1 (synthetic objects)

Start conservative, increase if overfitting!

══════════════════════════════════════════════════════════

6. OPTIMIZER SETTINGS
──────────────────────────────────────────────────────────
optimizer: 'AdamW' / 'SGD' / 'auto'
momentum: 0.937
weight_decay: 0.0005

AdamW (default, recommended):
- Adaptive learning rate
- Works well out of box
- Good for most cases

SGD (traditional):
- Requires more tuning
- Can be faster with right settings
- Use if you know what you're doing

'auto': Let YOLO choose (usually picks AdamW)

══════════════════════════════════════════════════════════

EXAMPLE CONFIGURATIONS:
══════════════════════════════════════════════════════════

CONSERVATIVE (Start here):
{
    'model': 'yolov8n.pt',
    'epochs': 50,
    'patience': 20,
    'batch': 16,
    'lr0': 0.01,
    'optimizer': 'auto',
    'mosaic': 1.0,
    'mixup': 0.0
}

AGGRESSIVE (If not overfitting):
{
    'model': 'yolov8s.pt',
    'epochs': 100,
    'patience': 30,
    'batch': 16,
    'lr0': 0.01,
    'optimizer': 'AdamW',
    'mosaic': 1.0,
    'mixup': 0.15,
    'copy_paste': 0.1
}

══════════════════════════════════════════════════════════
""")

print("✅ Exercise 1.2 Complete!")
print("\n📝 Start conservative, tune based on results!")


EXERCISE 1.2: Understanding Hyperparameters

YOLO HYPERPARAMETERS EXPLAINED:
══════════════════════════════════════════════════════════

1. MODEL SELECTION
──────────────────────────────────────────────────────────
model: 'yolov8n.pt' / 'yolov8s.pt' / 'yolov8m.pt'

Options:
- YOLOv8n (nano): 3.2M params, fastest, good for learning
- YOLOv8s (small): 11M params, balanced, production-ready
- YOLOv8m (medium): 25M params, slower, highest accuracy

When to upgrade:
- Start with 'n' (nano) for quick experiments
- Upgrade to 's' (small) when data quality is good
- Use 'm' (medium) only if 's' plateaus below target

══════════════════════════════════════════════════════════

2. TRAINING DURATION
──────────────────────────────────────────────────────────
epochs: 50-100
patience: 20-30

Epochs: How many times to see full dataset
- Too few (10): Underfitting, poor results
- Good (50-100): Balanced learning
- Too many (200+): Overfitting risk, waste time

Patience: Early stopping tolerance
- Sto

In [5]:
# ============================================================
# EXERCISE 1.3: TRAINING CONFIGURATION TEMPLATE
# ============================================================
print("\n" + "="*60)
print("EXERCISE 1.3: Training Configuration Template")
print("="*60)

# Example training configuration
training_config_template = {
    # === MODEL ===
    'model': 'yolov8n.pt',  # Start with nano
    
    # === DATA ===
    'data': 'path/to/data.yaml',  # Your dataset
    'imgsz': 640,  # Standard YOLO size
    
    # === TRAINING DURATION ===
    'epochs': 50,  # How long to train
    'patience': 20,  # Early stopping
    
    # === BATCH & WORKERS ===
    'batch': 16,  # Images per step
    'workers': 4,  # Data loading threads
    
    # === OPTIMIZATION ===
    'optimizer': 'auto',  # Let YOLO choose
    'lr0': 0.01,  # Initial learning rate
    'lrf': 0.01,  # Final learning rate
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3,
    
    # === AUGMENTATION ===
    'hsv_h': 0.015,  # Hue
    'hsv_s': 0.7,  # Saturation
    'hsv_v': 0.4,  # Brightness
    'degrees': 0.0,  # Rotation (0 for upright objects)
    'translate': 0.1,  # Translation
    'scale': 0.5,  # Scale
    'flipud': 0.0,  # No vertical flip
    'fliplr': 0.5,  # Horizontal flip 50%
    'mosaic': 1.0,  # Mosaic (always on)
    'mixup': 0.0,  # Mixup (off initially)
    
    # === VALIDATION ===
    'val': True,  # Validate during training
    'save': True,  # Save checkpoints
    'save_period': -1,  # Only save best
    
    # === OUTPUT ===
    'project': 'runs/detect',
    'name': 'my_detector',
    'exist_ok': False,
    
    # === DEVICE ===
    'device': 'cpu',  # Or 'cuda' for GPU
    
    # === LOGGING ===
    'verbose': True,
    'plots': True,
}

print("\n📋 Training Configuration Template:")
print("="*60)
for key, value in training_config_template.items():
    print(f"   {key}: {value}")

print("\n✅ Exercise 1.3 Complete!")
print("\n📝 Copy this template and modify for your project!")


EXERCISE 1.3: Training Configuration Template

📋 Training Configuration Template:
   model: yolov8n.pt
   data: path/to/data.yaml
   imgsz: 640
   epochs: 50
   patience: 20
   batch: 16
   workers: 4
   optimizer: auto
   lr0: 0.01
   lrf: 0.01
   momentum: 0.937
   weight_decay: 0.0005
   warmup_epochs: 3
   hsv_h: 0.015
   hsv_s: 0.7
   hsv_v: 0.4
   degrees: 0.0
   translate: 0.1
   scale: 0.5
   flipud: 0.0
   fliplr: 0.5
   mosaic: 1.0
   mixup: 0.0
   val: True
   save: True
   save_period: -1
   project: runs/detect
   name: my_detector
   exist_ok: False
   device: cpu
   verbose: True
   plots: True

✅ Exercise 1.3 Complete!

📝 Copy this template and modify for your project!


In [6]:
# ============================================================
# EXERCISE 2.1: UNDERSTANDING EVALUATION METRICS
# ============================================================
print("\n" + "="*60)
print("EXERCISE 2.1: Evaluation Metrics Explained")
print("="*60)

print("""
OBJECT DETECTION METRICS:
══════════════════════════════════════════════════════════

1. mAP@50 (Mean Average Precision at IoU=0.5)
──────────────────────────────────────────────────────────
MOST IMPORTANT METRIC!

What it measures:
- How accurate are your detections?
- At IoU threshold of 0.5 (50% overlap)

Range: 0.0 to 1.0 (0% to 100%)

Interpretation:
- 0.0-0.3 (0-30%): Poor, needs major improvement
- 0.3-0.5 (30-50%): Fair, needs more training/data
- 0.5-0.7 (50-70%): Good, usable for some applications
- 0.7-0.9 (70-90%): Excellent, production-ready
- 0.9-1.0 (90-100%): Outstanding, state-of-the-art

Example targets:
- Academic project: 50%+
- Production app: 70%+
- Safety-critical: 85%+

══════════════════════════════════════════════════════════

2. mAP@50-95 (mAP across IoU thresholds)
──────────────────────────────────────────────────────────
More strict metric!

What it measures:
- Average mAP from IoU=0.5 to IoU=0.95
- Requires more precise bounding boxes

Typical results:
- Usually 2-3x lower than mAP@50
- If mAP@50 = 75%, mAP@50-95 ≈ 50-55%

When to care:
- Academic papers (COCO benchmark)
- High-precision applications
- Comparing to published results

For most applications: Focus on mAP@50!

══════════════════════════════════════════════════════════

3. PRECISION
──────────────────────────────────────────────────────────
What it measures:
- Of all predictions, how many were correct?

Formula:
Precision = True Positives / (True Positives + False Positives)

High precision (0.8+):
- Few false alarms
- Model is "careful"
- Good for: Applications where false alarms are costly

Low precision (<0.5):
- Many false alarms
- Model is "trigger happy"
- Problem: Detects things that aren't there

Example:
- 100 detections made
- 80 were correct, 20 were wrong
- Precision = 80/100 = 0.8 (80%)

══════════════════════════════════════════════════════════

4. RECALL
──────────────────────────────────────────────────────────
What it measures:
- Of all true objects, how many did we find?

Formula:
Recall = True Positives / (True Positives + False Negatives)

High recall (0.8+):
- Finds most objects
- Model is "thorough"
- Good for: Safety applications (must catch everything)

Low recall (<0.5):
- Misses many objects
- Model is "blind" to some cases
- Problem: Objects not detected at all

Example:
- 100 objects exist in images
- Model found 75 of them
- Recall = 75/100 = 0.75 (75%)

══════════════════════════════════════════════════════════

5. PRECISION vs RECALL TRADEOFF
──────────────────────────────────────────────────────────
Can't maximize both simultaneously!

Lowering confidence threshold:
→ More detections (↑ Recall)
→ More false positives (↓ Precision)

Raising confidence threshold:
→ Fewer detections (↓ Recall)
→ Fewer false positives (↑ Precision)

Balance depends on application:

Safety-critical (construction, medical):
→ Prioritize Recall (catch everything)
→ Accept lower Precision (ok to have false alarms)

Performance-critical (real-time apps):
→ Balance Precision and Recall
→ Use confidence threshold tuning

Quality-critical (automated systems):
→ Prioritize Precision (minimize false alarms)
→ Accept lower Recall (ok to miss some)

══════════════════════════════════════════════════════════

6. CONFUSION MATRIX
──────────────────────────────────────────────────────────
Visual representation of classification errors

Diagonal = Correct predictions (want this dark/high)
Off-diagonal = Confusions/mistakes

Common patterns:
- helmet confused with no-helmet → Similar appearance
- person missed → Small objects hard to detect
- vest false positive → Background similar to vest

Use it to:
- Identify problematic class pairs
- Understand where model struggles
- Guide data collection priorities

══════════════════════════════════════════════════════════

PRACTICAL EXAMPLE:
══════════════════════════════════════════════════════════
Safety Equipment Detector Results:

v1 (Baseline):
- mAP@50: 17.1%
- Precision: 32%
- Recall: 32%
→ Analysis: Needs more training, balanced but low

v2 (Optimized):
- mAP@50: 48.5%
- Precision: 69%
- Recall: 27%
→ Analysis: Good precision, but missing objects (low recall)

v3 (Production):
- mAP@50: 75.1%
- Precision: 73.5%
- Recall: 72.1%
→ Analysis: Balanced and high, production-ready!

══════════════════════════════════════════════════════════
""")

print("✅ Exercise 2.1 Complete!")
print("\n📝 Key Metrics: mAP@50 (main), Precision, Recall!")


EXERCISE 2.1: Evaluation Metrics Explained

OBJECT DETECTION METRICS:
══════════════════════════════════════════════════════════

1. mAP@50 (Mean Average Precision at IoU=0.5)
──────────────────────────────────────────────────────────
MOST IMPORTANT METRIC!

What it measures:
- How accurate are your detections?
- At IoU threshold of 0.5 (50% overlap)

Range: 0.0 to 1.0 (0% to 100%)

Interpretation:
- 0.0-0.3 (0-30%): Poor, needs major improvement
- 0.3-0.5 (30-50%): Fair, needs more training/data
- 0.5-0.7 (50-70%): Good, usable for some applications
- 0.7-0.9 (70-90%): Excellent, production-ready
- 0.9-1.0 (90-100%): Outstanding, state-of-the-art

Example targets:
- Academic project: 50%+
- Production app: 70%+
- Safety-critical: 85%+

══════════════════════════════════════════════════════════

2. mAP@50-95 (mAP across IoU thresholds)
──────────────────────────────────────────────────────────
More strict metric!

What it measures:
- Average mAP from IoU=0.5 to IoU=0.95
- Requires mor

In [7]:
# ============================================================
# EXERCISE 2.2: MODEL COMPARISON BEST PRACTICES
# ============================================================
print("\n" + "="*60)
print("EXERCISE 2.2: Comparing Model Versions")
print("="*60)

print("""
HOW TO COMPARE MODELS:
══════════════════════════════════════════════════════════

1. KEEP TEST SET CONSTANT
──────────────────────────────────────────────────────────
CRITICAL: Always use same test images!

Why?
- Fair comparison
- Results are comparable
- Track true improvement

How?
- Split data once at start
- Never retrain on test set
- Never peek at test set during development

══════════════════════════════════════════════════════════

2. TRACK MULTIPLE METRICS
──────────────────────────────────────────────────────────
Don't just look at one number!

Track:
- mAP@50 (main metric)
- Precision (false alarm rate)
- Recall (miss rate)
- Per-class performance
- Training time
- Inference speed

Example comparison table:

| Version | mAP@50 | Precision | Recall | Train Time | Size |
|---------|--------|-----------|--------|------------|------|
| v1      | 17.1%  | 32%       | 32%    | 1 min      | 6MB  |
| v2      | 48.5%  | 69%       | 27%    | 10 min     | 6MB  |
| v3      | 75.1%  | 73.5%     | 72.1%  | 35 min     | 22MB |

══════════════════════════════════════════════════════════

3. ANALYZE PER-CLASS PERFORMANCE
──────────────────────────────────────────────────────────
Look at each class individually!

Example:

Class       | v1    | v2    | v3    | Improvement
------------|-------|-------|-------|-------------
person      | 78%   | 78%   | 51%   | Balanced better
helmet      | 0.1%  | 0.1%  | 57%   | HUGE gain!
vest        | 2.7%  | 2.7%  | 41%   | HUGE gain!
no-helmet   | 0%    | 0%    | 18%   | Now detectable
no-vest     | 1.6%  | 1.6%  | 21%   | Much better

Insights:
- v1-v2: Same data, no per-class improvement
- v2-v3: More data helped small objects (helmet/vest)
- Person decreased but acceptable (more balanced)

══════════════════════════════════════════════════════════

4. VISUALIZE RESULTS
──────────────────────────────────────────────────────────
Create comparison plots!

Useful visualizations:
1. Bar chart: Overall mAP across versions
2. Line chart: Training curves comparison
3. Heatmap: Per-class performance evolution
4. Confusion matrix: Error patterns

Benefits:
- Spot trends quickly
- Communicate to stakeholders
- Include in reports/presentations

══════════════════════════════════════════════════════════

5. DOCUMENT CHANGES
──────────────────────────────────────────────────────────
Keep detailed records!

For each version, document:
- Configuration changes
- Dataset changes
- Results
- Insights learned
- Next steps

Example log:

v1 → v2:
Changes:
- Epochs: 10 → 50
- Patience: 5 → 20
- Batch: 8 → 16

Results:
- mAP: 17.1% → 48.5% (+184%)
- Time: 1 min → 10 min

Insight:
- More training helped significantly
- Still plateauing, need more data

Next:
- Collect 3-4x more images
- Try bigger model (YOLOv8s)

══════════════════════════════════════════════════════════

6. KNOW WHEN TO STOP
──────────────────────────────────────────────────────────
Don't iterate forever!

Stop when:
✅ Met target performance (e.g., 70% mAP)
✅ Improvement < 2% between versions
✅ Good enough for application
✅ Time/resource constraints

Continue if:
❌ Below target by >5%
❌ Critical classes performing poorly
❌ Production requirements not met

Example decision:
- Target: 70% mAP
- v3 result: 75.1% mAP
- Decision: ✅ STOP! Deploy v3
- Reasoning: Exceeded target, production-ready

══════════════════════════════════════════════════════════
""")

print("✅ Exercise 2.2 Complete!")
print("\n📝 Track metrics, visualize, document, know when to stop!")


EXERCISE 2.2: Comparing Model Versions

HOW TO COMPARE MODELS:
══════════════════════════════════════════════════════════

1. KEEP TEST SET CONSTANT
──────────────────────────────────────────────────────────
CRITICAL: Always use same test images!

Why?
- Fair comparison
- Results are comparable
- Track true improvement

How?
- Split data once at start
- Never retrain on test set
- Never peek at test set during development

══════════════════════════════════════════════════════════

2. TRACK MULTIPLE METRICS
──────────────────────────────────────────────────────────
Don't just look at one number!

Track:
- mAP@50 (main metric)
- Precision (false alarm rate)
- Recall (miss rate)
- Per-class performance
- Training time
- Inference speed

Example comparison table:

| Version | mAP@50 | Precision | Recall | Train Time | Size |
|---------|--------|-----------|--------|------------|------|
| v1      | 17.1%  | 32%       | 32%    | 1 min      | 6MB  |
| v2      | 48.5%  | 69%       | 27%    |

In [8]:
# ============================================================
# EXERCISE 2.3: TROUBLESHOOTING COMMON ISSUES
# ============================================================
print("\n" + "="*60)
print("EXERCISE 2.3: Common Training Issues & Solutions")
print("="*60)

print("""
TROUBLESHOOTING GUIDE:
══════════════════════════════════════════════════════════

ISSUE 1: Loss not decreasing
──────────────────────────────────────────────────────────
Symptoms:
- Loss stays flat or increases
- mAP stays at 0% or very low

Possible causes & solutions:
❌ Learning rate too high
   → Reduce lr0 from 0.01 to 0.001

❌ Bad annotations
   → Check labels visually
   → Verify coordinates are normalized (0-1)

❌ Wrong data.yaml paths
   → Verify paths are correct
   → Check images/labels exist

❌ Too few epochs
   → Increase from 10 to 50+

══════════════════════════════════════════════════════════

ISSUE 2: Overfitting (train good, test bad)
──────────────────────────────────────────────────────────
Symptoms:
- Training loss keeps decreasing
- Validation loss increases or plateaus
- mAP on train > mAP on test by >20%

Solutions:
✅ Add more augmentation
   → Increase mosaic, mixup, fliplr

✅ Collect more data
   → Need 2-3x more images

✅ Add regularization
   → Increase weight_decay to 0.001

✅ Early stopping
   → Reduce patience to stop sooner

══════════════════════════════════════════════════════════

ISSUE 3: Underfitting (both train and test poor)
──────────────────────────────────────────────────────────
Symptoms:
- Low performance on both train and test
- Loss plateaus early
- mAP < 30% on training set

Solutions:
✅ Train longer
   → Increase epochs to 100+

✅ Use bigger model
   → Upgrade from YOLOv8n to YOLOv8s

✅ Reduce augmentation
   → Temporarily disable mosaic/mixup

✅ Check data quality
   → Verify annotations are correct

══════════════════════════════════════════════════════════

ISSUE 4: Some classes not learning
──────────────────────────────────────────────────────────
Symptoms:
- One class has 0% or <5% mAP
- Other classes perform well
- Class shows in confusion matrix but never predicted

Solutions:
✅ Check class balance
   → Need at least 50-100 examples per class

✅ Collect more examples
   → Focus on underperforming class

✅ Check annotation quality
   → Verify boxes are tight and accurate

✅ Train longer
   → Smaller objects need more epochs

══════════════════════════════════════════════════════════

ISSUE 5: Training crashes or stops
──────────────────────────────────────────────────────────
Symptoms:
- Training aborts with error
- Out of memory
- Process killed

Solutions:
✅ Reduce batch size
   → Try batch=8 or batch=4

✅ Close other programs
   → Free up RAM/VRAM

✅ Check disk space
   → Need 1-2GB free for checkpoints

✅ Update ultralytics
   → pip install --upgrade ultralytics

══════════════════════════════════════════════════════════

ISSUE 6: Training too slow
──────────────────────────────────────────────────────────
Symptoms:
- Each epoch takes >5 minutes
- Training would take hours

Solutions:
✅ Use GPU if available
   → device='cuda' instead of 'cpu'
   → 5-10x speedup!

✅ Reduce workers
   → workers=2 or workers=0
   → If CPU bottlenecked

✅ Use smaller model
   → YOLOv8n instead of YOLOv8s
   → For quick experiments

✅ Reduce image size
   → imgsz=416 instead of 640
   → Faster but less accurate

══════════════════════════════════════════════════════════

DIAGNOSTIC CHECKLIST:
══════════════════════════════════════════════════════════
Before asking for help, check:

✓ Dataset structure correct (train/valid/test)
✓ data.yaml paths are correct
✓ Images and labels paired correctly
✓ Coordinates normalized (0-1)
✓ Class IDs start from 0
✓ At least 50 images per class
✓ Ultralytics version up to date
✓ Enough disk space (1GB+)
✓ Tried training for 50+ epochs
✓ Checked tensorboard logs

══════════════════════════════════════════════════════════
""")

print("✅ Exercise 2.3 Complete!")
print("\n📝 Most issues fixed by: more data, more epochs, or smaller batch!")


EXERCISE 2.3: Common Training Issues & Solutions

TROUBLESHOOTING GUIDE:
══════════════════════════════════════════════════════════

ISSUE 1: Loss not decreasing
──────────────────────────────────────────────────────────
Symptoms:
- Loss stays flat or increases
- mAP stays at 0% or very low

Possible causes & solutions:
❌ Learning rate too high
   → Reduce lr0 from 0.01 to 0.001

❌ Bad annotations
   → Check labels visually
   → Verify coordinates are normalized (0-1)

❌ Wrong data.yaml paths
   → Verify paths are correct
   → Check images/labels exist

❌ Too few epochs
   → Increase from 10 to 50+

══════════════════════════════════════════════════════════

ISSUE 2: Overfitting (train good, test bad)
──────────────────────────────────────────────────────────
Symptoms:
- Training loss keeps decreasing
- Validation loss increases or plateaus
- mAP on train > mAP on test by >20%

Solutions:
✅ Add more augmentation
   → Increase mosaic, mixup, fliplr

✅ Collect more data
   → Need 2-3x m

In [11]:
# ============================================================
# DAY 13 COMPLETE: FINE-TUNING & OPTIMIZATION
# ============================================================
print("\n" + "="*60)
print("🎉 DAY 13 COMPLETE!")
print("="*60)

print("""
══════════════════════════════════════════════════════════
DAY 13 COMPLETE: FINE-TUNING & OPTIMIZATION
══════════════════════════════════════════════════════════
Date: October 27, 2025
Time Investment: ~4 hours
EXERCISES COMPLETED: 2 parts

──────────────────────────────────────────────────────────
Part 1: Training Strategy (✅ Complete)
├─ Exercise 1.1: Model iteration strategy
├─ Exercise 1.2: Hyperparameter tuning
├─ Exercise 1.3: Training configuration template
└─ Learning: v1→v2→v3 methodology

Part 2: Evaluation & Analysis (✅ Complete)
├─ Exercise 2.1: Evaluation metrics explained
├─ Exercise 2.2: Model comparison methodology
├─ Exercise 2.3: Troubleshooting common issues
└─ Learning: mAP, precision, recall, debugging

KEY LEARNINGS:
──────────────
🧠 Iteration Strategy:
   • v1: Quick baseline (identify issues)
   • v2: Optimize hyperparameters
   • v3: Scale up (data + model)
   • Don't try to perfect v1!

🧠 Hyperparameters:
   • Start conservative
   • Epochs: 50-100 for small datasets
   • Batch: 16 is safe default
   • Learning rate: 0.01 works well
   • Augmentation: mosaic always on

🧠 Evaluation Metrics:
   • mAP@50: Main metric (aim for 70%+)
   • Precision: Low false alarms
   • Recall: Catch all objects
   • Balance depends on application

🧠 Troubleshooting:
   • Most issues: more data or more epochs
   • Overfitting: more augmentation
   • Slow classes: need more examples
   • Crashes: reduce batch size

PRODUCTION WORKFLOW:
────────────────────
1. Baseline (v1): Quick test, 10 epochs
2. Analyze: What's working? What's not?
3. Optimize (v2): Better hyperparameters, 50 epochs
4. Analyze: Still below target?
5. Scale (v3): More data + bigger model
6. Deploy: Above target? Ship it!

READY FOR:
──────────
✅ Training your own YOLO models
✅ Iterating systematically
✅ Evaluating results properly
✅ Debugging training issues
✅ Reaching production targets

Week 2 Progress: 86% (6/7 days)
Overall Progress: 7.7% (13/168 days)

STATUS: DAY 13 COMPLETE! 🎉
NEXT: Day 14 - Week 2 Review & Consolidation
""")

print("\n💪 I Just did YOLO training!")
print("\n🚀 Tomorrow: Week 2 review and wrap-up!")


🎉 DAY 13 COMPLETE!

══════════════════════════════════════════════════════════
DAY 13 COMPLETE: FINE-TUNING & OPTIMIZATION
══════════════════════════════════════════════════════════
Date: October 27, 2025
Time Investment: ~4 hours
EXERCISES COMPLETED: 2 parts

──────────────────────────────────────────────────────────
Part 1: Training Strategy (✅ Complete)
├─ Exercise 1.1: Model iteration strategy
├─ Exercise 1.2: Hyperparameter tuning
├─ Exercise 1.3: Training configuration template
└─ Learning: v1→v2→v3 methodology

Part 2: Evaluation & Analysis (✅ Complete)
├─ Exercise 2.1: Evaluation metrics explained
├─ Exercise 2.2: Model comparison methodology
├─ Exercise 2.3: Troubleshooting common issues
└─ Learning: mAP, precision, recall, debugging

KEY LEARNINGS:
──────────────
🧠 Iteration Strategy:
   • v1: Quick baseline (identify issues)
   • v2: Optimize hyperparameters
   • v3: Scale up (data + model)
   • Don't try to perfect v1!

🧠 Hyperparameters:
   • Start conservative
   • Epoch